This implementation is used to generate a single NumPy array file for both the training and testing datasets. The NumPy array contains images of single digits, created from coordinates obtained through metadata files. These metadata files provide the digit coordinates within the images.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
import os

Remove Multidigit files. digitStruct.csv is created using the following Github source code - https://github.com/prijip/Py-Gsvhn-DigitStruct-Reader.git



In [9]:
def removeMultiDigitFilesAndCreateNPYarray(imageFilepath, outputFilename):
    
    CSVFilename = imageFilepath + "\\digitStruct.csv"
    # Define the rectangle coordinates (x, y, width, height)
    coordinates = pd.read_csv(CSVFilename)
    
    # Find all unique strings in the first column that occur more than once
    value_counts = coordinates['FileName'].value_counts()
    to_remove = value_counts[value_counts > 1].index.tolist()    
   
    # Remove rows where the first column matches any string in 'to_remove'
    filtered_coordinates = coordinates[~coordinates['FileName'].isin(to_remove)]    
    
    # Get the rows where we have multiple digit files
    multi_digit_files = coordinates[coordinates['FileName'].isin(to_remove)]
    
    # Save multidigit file names and labels in this file
    np.save(outputFilename +'_multipleDigitPNG.npy', multi_digit_files)
    
    # Initialize an empty list to store image arrays
    image_arrays = []
    image_label_arrays = []
    
    # Loop through each file, load the image, and append its array to the list
    for index, row in filtered_coordinates.iterrows():
        if row['DigitLabel'] != 10:
            image_path = os.path.join(imageFilepath, row['FileName'])
            img = cv2.imread(image_path)
           
            resized_image = cv2.resize(img, (32, 32), interpolation=cv2.INTER_AREA)
        
            image_array = np.array(resized_image)
            image_arrays.append(image_array)
            
            # image_label_array = np.array(row['DigitLabel'])
            image_label_arrays.append(row['DigitLabel'])
    
    # Stack all image arrays into a single NumPy array
    all_images_array = np.stack(image_arrays)
    
    # Save the NumPy array to a file
    np.save(outputFilename +'_images.npy', all_images_array)
    np.save(outputFilename + '_label.npy', image_label_arrays)
   

Create a new single digit image file by splitting the image from multi-digit image.

In [10]:
def segmentImagesAndSaveNPYarray(ImageFilePath, multi_images_NPY, target):
    mult_image_files = np.load(multi_images_NPY, allow_pickle=True)
    image_list = []
    image_label_list = []
    for i in np.arange(len(mult_image_files)):
        try:
            # Load the original image
            image = cv2.imread(ImageFilePath + "\\" + mult_image_files[i,0])  
            coordinates = mult_image_files[i,2:]            
            x=coordinates[0]
            y=coordinates[1]
            w=coordinates[2]
            h=coordinates[3]        
    
            # Crop the image using array slicing
            cropped_image = image[y:y+h, x:x+w]
            
            # Check if the image was loaded successfully
            if cropped_image is None:
                continue
            resized_image = cv2.resize(cropped_image, (32, 32))
            
            img_array = np.array(resized_image)
            image_list.append(img_array)
            image_label_list.append(mult_image_files[i,1])
        except cv2.error as e:
            print(f"Error resizing image at {mult_image_files[i,0]}: {e}. Skipping.")
            continue
       
    # Convert the List to a Numpy Array
    image_array = np.stack(image_list)  
    image_label_array = np.stack(image_label_list)  
    
    # Save the cropped images and labels
    np.save(target + "_images_singleImages.npy", image_array)
    np.save(target + "_labels_singleImages.npy", image_label_array)
 

Plot segmented image

In [11]:
def plotSegmentedImage(imageName):          
    # Load the image
    image = cv2.imread(imageName)    
    plt.imshow(image, cmap='gray')
    plt.show()

Save image in a file

In [12]:
def testSavedImages(plotTest_images,plot_Test_labels,n,target):
    _, axes = plt.subplots(nrows=1, ncols = n, figsize=(16,4))
    for ax, image, label in zip(axes, plotTest_images[0:n], plot_Test_labels):
        ax.set_axis_off()
        ax.imshow(image, cmap='gray')        
        ax.set_title(target + "%i" % label)

Set the ImageFilePath where downloaded images are stored. Replace the string in ImageFilePath by the folder pathe where 
downloaded images are stored.

In [17]:
ImageFilePath = ".\\temp\\"
multi_images_NPY = 'temp_multipleDigitPNG.npy'
target = ".\\temp\\temp"

Call to remove multi-digit image files

In [ ]:
removeMultiDigitFilesAndCreateNPYarray(ImageFilePath, target)

Test the newly created numpy array files contaning image data

In [ ]:
test_images_ = np.load(target + '_images.npy')
test_labels_ = np.load(target + '_label.npy')
n = 10
_, axes = plt.subplots(nrows=1, ncols = 10, figsize=(16,4))
for ax, image, label in zip(axes, test_images_[0:n], test_labels_):
    ax.set_axis_off()
    ax.imshow(image, cmap='gray')
    
    ax.set_title(target + ": %i" % label)

Store the single image numpy arrays as a seperate file. These files will be input to the digit classification algorithms.

In [ ]:
segmentImagesAndSaveNPYarray(ImageFilePath, multi_images_NPY, target)

Test the single digit numpy array files.

In [ ]:
test_images = np.load(target+'_images_singleImages.npy')
test_labels = np.load(target+'_labels_singleImages.npy')
testSavedImages(test_images,test_labels,5,target)